In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os

In [2]:
# -----------------------------------------------------
# OPEN FILE
# -----------------------------------------------------
filename="shape_coleta.gpkg"
path='/home/akel/PycharmProjects/Data_ambiental/data/process/'
arquivo = os.path.join(path, filename)

gdf = gpd.read_file(arquivo)
gdf_t=gdf[['setor','geometry']].copy()

gdf_t['n_dias']=gdf['dias'].apply(lambda x: len(eval(x))) #criando uma coluna com numerodias

filename="shape_bairros.gpkg"
arquivo = os.path.join(path, filename)
gdf_b = gpd.read_file(arquivo)
gdf_b.rename(columns={"NM_BAIRRO": "Bairro"}, inplace=True)


In [3]:
gdf_b.head(3)

,Bairro,geometry
0,Marco,"MULTIPOLYGON (((-48.45072 -1.43336, -48.45066 ..."
1,Reduto,"MULTIPOLYGON (((-48.48838 -1.44583, -48.48836 ..."
2,Batista Campos,"MULTIPOLYGON (((-48.4884 -1.46606, -48.48916 -..."


In [3]:
gdf_t = gdf_t.to_crs(gdf_b.crs)


gdf_b = gdf_b.to_crs(epsg=3857)
gdf_t = gdf_t.to_crs(epsg=3857)
print(gdf_b.crs)
print(gdf_t.crs)

EPSG:3857
EPSG:3857


In [8]:
# bairro_select= gdf_b[gdf_b['Bairro'] == 'Condor']

# setores_no_bairro= gdf_t[gdf_t.intersects(bairro_select.iloc[0].geometry)]
gdf_b['Bairro'].tolist #lista de Bairros

<bound method IndexOpsMixin.tolist of 0              Marco
1             Reduto
2     Batista Campos
3       Cidade Velha
4              Guamá
           ...      
66        Sacramenta
67      Maracangalha
68          Pedreira
69          Barreiro
70           Miramar
Name: Bairro, Length: 71, dtype: object>

In [5]:
gdf_b['Bairro'].tolist #lista de Bairros

bairro_select= gdf_b[gdf_b['Bairro'] == 'Marambaia']

setores_no_bairro= gdf_t[gdf_t.intersects(bairro_select.iloc[0].geometry)].copy()

# # Seleciona o bairro desejado
bairro_geom = bairro_select.iloc[0].geometry

# Inicializa uma lista para armazenar os percentuais
percentuais = []

# Loop sobre cada setor
for setor_geom in setores_no_bairro.geometry:
    # Calcula a área de interseção com o bairro
    intersecao = setor_geom.intersection(bairro_geom)
    # Calcula o percentual coberto do setor
    percentual = (intersecao.area / setor_geom.area) * 100
    # Adiciona à lista
    percentuais.append(percentual)

# Adiciona a lista como nova coluna no GeoDataFrame
setores_no_bairro['percentual_coberto'] = np.round(percentuais,2)

# Mostrar o setor e o percentual
setores_no_bairro[['setor', 'percentual_coberto', 'n_dias']]


,setor,percentual_coberto,n_dias
42,1910,2.00,3
56,1812,100.00,3
57,1807,98.32,3
59,1810,0.10,3
97,1206,0.87,3
100,1806,97.11,3
101,1811,97.51,3
102,1906,0.94,3


In [10]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
from shapely.validation import make_valid

# -----------------------------------------------------
# OPEN FILE
# -----------------------------------------------------
filename = "shape_coleta.gpkg"
path = '/home/akel/PycharmProjects/Data_ambiental/data/process/'
arquivo = os.path.join(path, filename)

gdf = gpd.read_file(arquivo)
gdf_t = gdf[['setor','geometry']].copy()
gdf_t['n_dias'] = gdf['dias'].apply(lambda x: len(eval(x)))

filename = "shape_bairros.gpkg"
arquivo = os.path.join(path, filename)
gdf_b = gpd.read_file(arquivo)
gdf_b.rename(columns={"NM_BAIRRO": "Bairro"}, inplace=True)

# Converter para o mesmo CRS
gdf_t = gdf_t.to_crs(gdf_b.crs)
gdf_b = gdf_b.to_crs(epsg=3857)
gdf_t = gdf_t.to_crs(epsg=3857)

# Função para corrigir geometrias inválidas
def corrigir_geometria(geom):
    if not geom.is_valid:
        return make_valid(geom)
    return geom

# Aplicar correção às geometrias
gdf_b['geometry'] = gdf_b['geometry'].apply(corrigir_geometria)
gdf_t['geometry'] = gdf_t['geometry'].apply(corrigir_geometria)

# Lista para armazenar resultados
resultados = []

# Processar cada bairro
for bairro_nome in gdf_b['Bairro'].unique():
    print(f"Processando bairro: {bairro_nome}")
    
    try:
        # Selecionar o bairro atual
        bairro_select = gdf_b[gdf_b['Bairro'] == bairro_nome]
        
        if len(bairro_select) == 0:
            continue
            
        # Geometria do bairro (corrigida se necessário)
        bairro_geom = corrigir_geometria(bairro_select.iloc[0].geometry)
        
        # Encontrar setores que intersectam com o bairro
        setores_no_bairro = gdf_t[gdf_t.intersects(bairro_geom)].copy()
        
        if len(setores_no_bairro) == 0:
            print(f"  Nenhum setor encontrado para o bairro {bairro_nome}")
            resultados.append({
                'Bairro': bairro_nome,
                'n_dias_medio_ponderado': 0,
                'quantidade_setores': 0,
                'observacao': 'Nenhum setor intersecta'
            })
            continue
        
        # Calcular percentuais de cobertura com tratamento de erro
        percentuais = []
        setores_validos = []
        
        for idx, row in setores_no_bairro.iterrows():
            try:
                setor_geom = corrigir_geometria(row.geometry)
                
                # Verificar se a geometria é válida e não vazia
                if setor_geom.is_empty or setor_geom.area == 0:
                    percentuais.append(0)
                    setores_validos.append(False)
                    continue
                
                # Calcular interseção
                intersecao = setor_geom.intersection(bairro_geom)
                
                # Se a interseção for vazia, percentual é 0
                if intersecao.is_empty:
                    percentual = 0
                else:
                    percentual = (intersecao.area / setor_geom.area) * 100
                
                percentuais.append(percentual)
                setores_validos.append(True)
                
            except Exception as e:
                print(f"  Erro no setor {row['setor']}: {e}")
                percentuais.append(0)
                setores_validos.append(False)
        
        # Adicionar percentuais ao DataFrame
        setores_no_bairro = setores_no_bairro.copy()
        setores_no_bairro['percentual_coberto'] = np.round(percentuais, 2)
        setores_no_bairro['valido'] = setores_validos
        
        # Filtrar apenas setores válidos para o cálculo
        setores_validos_df = setores_no_bairro[setores_no_bairro['valido']]
        
        if len(setores_validos_df) == 0:
            print(f"  Nenhum setor válido para cálculo no bairro {bairro_nome}")
            resultados.append({
                'Bairro': bairro_nome,
                'n_dias_medio_ponderado': 0,
                'quantidade_setores': 0,
                'quantidade_setores_validos': 0,
                'observacao': 'Nenhum setor válido'
            })
            continue
        
        # Calcular média ponderada do n_dias
        n_dias_values = setores_validos_df['n_dias'].values
        pesos = setores_validos_df['percentual_coberto'].values / 100  # Converter para proporção
        
        # Filtrar pesos e valores onde peso > 0
        mask = pesos > 0
        if sum(mask) > 0:
            n_dias_filtrado = n_dias_values[mask]
            pesos_filtrado = pesos[mask]
            media_ponderada = np.average(n_dias_filtrado, weights=pesos_filtrado)
        else:
            # Se todos os pesos forem zero, usar média simples dos setores válidos
            media_ponderada = np.mean(n_dias_values)
        
        # Adicionar à lista de resultados
        resultados.append({
            'Bairro': bairro_nome,
            'n_dias_medio_ponderado': round(media_ponderada, 2),
            'quantidade_setores': len(setores_no_bairro),
            'quantidade_setores_validos': len(setores_validos_df),
            'observacao': 'OK'
        })
        
        print(f"  ✓ Processado: {len(setores_validos_df)} setores válidos, média: {round(media_ponderada, 2)}")
        
    except Exception as e:
        print(f"  ✗ Erro ao processar bairro {bairro_nome}: {e}")
        resultados.append({
            'Bairro': bairro_nome,
            'n_dias_medio_ponderado': 0,
            'quantidade_setores': 0,
            'quantidade_setores_validos': 0,
            'observacao': f'Erro: {str(e)}'
        })

# Criar DataFrame com os resultados
df_resultados = pd.DataFrame(resultados)

# Ordenar por nome do bairro
df_resultados = df_resultados.sort_values('Bairro').reset_index(drop=True)

# Mostrar resultados
print("\n" + "="*80)
print("RESULTADO FINAL: n_dias médio ponderado para cada bairro")
print("="*80)
print(df_resultados[['Bairro', 'n_dias_medio_ponderado', 'quantidade_setores_validos', 'observacao']])

# Estatísticas resumidas
total_bairros = len(df_resultados)
bairros_com_dados = len(df_resultados[df_resultados['quantidade_setores_validos'] > 0])
print(f"\nESTATÍSTICAS:")
print(f"Total de bairros processados: {total_bairros}")
print(f"Bairros com dados válidos: {bairros_com_dados}")
print(f"Bairros sem dados: {total_bairros - bairros_com_dados}")

# Lista simplificada para exportação
lista_final = df_resultados[['Bairro', 'n_dias_medio_ponderado']].to_dict('records')
print(f"\nLista final com {len(lista_final)} bairros:")
for resultado in lista_final:
    if resultado['n_dias_medio_ponderado'] > 0:
        print(f"  {resultado['Bairro']}: {resultado['n_dias_medio_ponderado']} dias")

Processando bairro: Marco
  ✓ Processado: 13 setores válidos, média: 5.13
Processando bairro: Reduto
  ✓ Processado: 2 setores válidos, média: 6.0
Processando bairro: Batista Campos
  ✓ Processado: 9 setores válidos, média: 5.74
Processando bairro: Cidade Velha
  ✓ Processado: 4 setores válidos, média: 4.8
Processando bairro: Guamá
  ✓ Processado: 10 setores válidos, média: 4.63
Processando bairro: Campina
  ✓ Processado: 7 setores válidos, média: 6.0
Processando bairro: São Brás
  ✓ Processado: 4 setores válidos, média: 5.36
Processando bairro: Umarizal
  ✓ Processado: 10 setores válidos, média: 6.0
Processando bairro: Nazaré
  ✓ Processado: 6 setores válidos, média: 6.0
Processando bairro: Cremação
  ✓ Processado: 4 setores válidos, média: 4.21
Processando bairro: Tapanã
  ✓ Processado: 9 setores válidos, média: 3.01
Processando bairro: Bengui
  ✓ Processado: 4 setores válidos, média: 3.0
Processando bairro: Coqueiro
  ✓ Processado: 12 setores válidos, média: 3.01
Processando bairro:

In [19]:
df_resultados[df_resultados['Bairro']=='Batista Campos']

,Bairro,n_dias_medio_ponderado,quantidade_setores,quantidade_setores_validos,observacao
5,Batista Campos,6.0,9,9.0,OK
